In [17]:
%%file Conv_transpose2d.py
import torch
import torch.nn.functional as F
import numpy as np

def conv_transpose2d(in_channels, out_channels, kernel_size, stride=1, groups=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros'):
    def convolution_transpose2d(input_m):
        if bias:
            value_b = torch.rand(out_channels)
        else:
            value_b = torch.zeros(out_channels)

        # Проверяем правила для свертки с группами
        assert in_channels % groups == 0
        assert out_channels % groups == 0

        if padding_mode == 'reflect' or padding_mode == 'replicate' or padding_mode == 'circular':
            raise ValueError("Unsupported padding_mode")

        if type(kernel_size) == tuple:
            filter = torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
        elif type(kernel_size) == int:
            filter = torch.rand(in_channels, out_channels, kernel_size, kernel_size)
        else:
            raise ValueError("Unsupported kernel_size type")

        out_tensor = []
        for l in range(out_channels):
            f = np.array([])
            for i in range (0, input_m.shape[1] - ((filter.shape[2]-1) * dilation + 1) + 1, stride):
                for j in range (0, input_m.shape[2] - ((filter.shape[3]-1) * dilation + 1) + 1, stride):
                    s = 0
                    for c in range (in_channels//groups):
                        if groups > 1:
                            val = input_m[l * (in_channels//groups) + c][i:i + (filter.shape[2]-1) * dilation + 1:dilation, j:j + (filter.shape[3]-1) * dilation + 1:dilation]
                        else:
                            val = input_m[c][i:i + (filter.shape[2]-1) * dilation + 1:dilation, j:j + (filter.shape[3] - 1) * dilation + 1:dilation]
                        mini_sum = (val * filter[l][c]).sum()
                        s = s + mini_sum
                    f = np.append(f, float(s + value_b[l]))
            out_tensor.append(torch.tensor(f, dtype=torch.float).view(1, 1, -1))
        return np.array(out_tensor), torch.tensor(np.array(filter)), torch.tensor(np.array(value_b))
    return convolution_transpose2d

Overwriting Conv_transpose2d.py


In [18]:
%%file test_conv_transpose2d.py
import unittest
import torch
import torch.nn as nn
from Conv_transpose2d import conv_transpose2d

class TestConvolutionTranspose2D(unittest.TestCase):
    def test_conv_transpose2d_1(self):
        tensor = torch.rand(2, 10, 10)

        ConvTranspose2D = conv_transpose2d(in_channels=2, out_channels=2, kernel_size=2, stride=10, padding=0, output_padding=0, bias=True, dilation=3, padding_mode='zeros')
        result, kernel_size, bias = ConvTranspose2D(tensor)
        
        torchFunction = nn.ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, stride=10, padding=0, output_padding=0, bias=True, dilation=3, padding_mode='zeros')
        torchFunction.weight.data = torch.tensor(kernel_size)
        torchFunction.bias.data = torch.tensor(bias)

    def test_conv_transpose2d_2(self):
        tensor = torch.rand(1, 1, 1)

        ConvTranspose2D = conv_transpose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, output_padding=0, bias=True, dilation=3, padding_mode='zeros')
        result, kernel_size, bias = ConvTranspose2D(tensor)
        
        torchFunction = nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, output_padding=0, bias=True, dilation=3, padding_mode='zeros')
        torchFunction.weight.data = torch.tensor(kernel_size)
        torchFunction.bias.data = torch.tensor(bias)

Overwriting test_conv_transpose2d.py


In [19]:
!python -m unittest test_conv_transpose2d.py

d:\SMZ\lab3\Conv_transpose2d.py:41: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return np.array(out_tensor), torch.tensor(np.array(filter)), torch.tensor(np.array(value_b))
d:\SMZ\lab3\Conv_transpose2d.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(out_tensor), torch.tensor(np.array(filter)), torch.tensor(np.array(value_b))
d:\SMZ\lab3\test_c

In [66]:
%%file Convolution2D_dop.py
import torch
import torch.nn.functional as F
import numpy as np

def cv2d_dop(in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
    def convolution2d_dop(input_m):
        if bias:
            value_b = torch.rand(out_channels)
        else:
            value_b = torch.zeros(out_channels)

        # Проверяем правила для свертки с группами
        assert in_channels % groups == 0
        assert out_channels % groups == 0

        if padding_mode == 'zeros':
            input_m = F.pad(input_m, (padding, padding, padding, padding), mode='constant', value=0)
        elif padding_mode == 'reflect':
            input_m = F.pad(input_m, (padding, padding, padding, padding), mode='reflect')
        elif padding_mode == 'replicate':
            input_m = F.pad(input_m, (padding, padding, padding, padding), mode='replicate')
        elif padding_mode == 'circular':
            input_m = circular_pad(input_m, padding)
        else:
            raise ValueError("Unsupported padding_mode")

        if type(kernel_size) == tuple:
            filter = torch.rand(out_channels, in_channels // groups, kernel_size[0], kernel_size[1])
        elif type(kernel_size) == int:
            filter = torch.rand(out_channels, in_channels // groups, kernel_size, kernel_size)
        else:
            raise ValueError("Unsupported kernel_size type")
        
        stride = 1
        result_matrix = []
        for matr in input_m:
            # Увеличиваем выборку входной матрицы с помощью transp_stride
            upsampled_matr = np.kron(matr, np.ones((transp_stride, transp_stride)))
            # Дополняем матрицу с увеличенной дискретизацией
            pad = kernel_size - 1
            pad_matr = np.pad(upsampled_matr, pad_width=pad, mode='constant')
            result_matrix.append(pad_matr)
        input_m = torch.tensor(result_matrix, dtype=torch.float)
        
        filter_np = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))
        filter_tensor = torch.tensor(filter_np)
        filter_for_transpose = torch.flip(filter_tensor, dims=[2, 3]).permute(1, 0, 2, 3)

        out_tensor = []
        for l in range(out_channels):
            f = np.array([])
            for i in range(0, input_m.shape[1] - ((filter.shape[2]-1) * dilation + 1) + 1, stride):
                for j in range(0, input_m.shape[2] - ((filter.shape[3]-1) * dilation + 1) + 1, stride):
                    s = 0
                    for c in range(in_channels // groups):
                        if groups > 1:
                            val = input_m[l * (in_channels // groups) + c][i:i + (filter.shape[2]-1) * dilation + 1:dilation, j:j + (filter.shape[3]-1) * dilation + 1:dilation]
                        else:
                            val = input_m[c][i:i + (filter.shape[2]-1) * dilation + 1:dilation, j:j + (filter.shape[3] - 1) * dilation + 1:dilation]
                        mini_sum = (val * filter[l][c]).sum()
                        s = s + mini_sum
                    f = np.append(f, float(s + value_b[l]))
            out_tensor.append(torch.tensor(f, dtype=torch.float).view(1, 1, -1))

        out_tensor_np = np.array([tensor.numpy() for tensor in out_tensor])
        return out_tensor_np, filter_for_transpose, torch.tensor(np.array(value_b))

    return convolution2d_dop


Overwriting Convolution2D_dop.py


In [67]:
%%file test_conv_transpose2d_dop.py
import unittest
import torch
import torch.nn as nn
from Convolution2D_dop import cv2d_dop

class TestConvolutionTranspose2D_dop(unittest.TestCase):
    def test_conv_transpose2d_dop_1(self):
        tensor = torch.rand(2, 10, 10)

        ConvTranspose2D = cv2d_dop(in_channels=2, out_channels=2, kernel_size=2, transp_stride=10, bias=True)
        result, kernel_size, bias = ConvTranspose2D(tensor)
        
        torchFunction = nn.ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, stride=10, bias=True)
        torchFunction.weight.data = torch.tensor(kernel_size)
        torchFunction.bias.data = torch.tensor(bias)

    def test_conv_transpose2d_dop_2(self):
        tensor = torch.rand(1, 1, 1)

        ConvTranspose2D = cv2d_dop(in_channels=1, out_channels=1, kernel_size=1, transp_stride=1, bias=True)
        result, kernel_size, bias = ConvTranspose2D(tensor)
        
        torchFunction = nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, bias=True)
        torchFunction.weight.data = torch.tensor(kernel_size)
        torchFunction.bias.data = torch.tensor(bias)

Overwriting test_conv_transpose2d_dop.py


In [68]:
!python -m unittest test_conv_transpose2d_dop.py

d:\SMZ\lab3\Convolution2D_dop.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  input_m = torch.tensor(result_matrix, dtype=torch.float)
d:\SMZ\lab3\test_conv_transpose2d_dop.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torchFunction.weight.data = torch.tensor(kernel_size)
d:\SMZ\lab3\test_conv_transpose2d_dop.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torchFunction.bias.data = torch.tensor(bias)
